In [ ]:
import torch

import torchvision
from torchvision import datasets, models, transforms

import numpy as np
import matplotlib.pyplot as plt
import os

## Veri artırma 

In [ ]:

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()
    ]),
}

## Veri kümesini yükleme
<p>torchvision kütüphanesi bize birçok veri kümesini hazır bir şekilde sağlar
</p>
<p>DOC: https://pytorch.org/docs/stable/torchvision/datasets.html </p>
<p>Ama bu hafta kendi veri kümemizi kullanacağız.</p>

In [ ]:
data_dir = 'Data splitted/'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

## CNN (Convolutional Neural Networks) Modelinin Oluşturulması

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()    
        
        # Katmanları gruplama: nn.Sequential()
        self.features = nn.Sequential(
            # Convolutional Layer
            # conv2d(in channel, out channel, kernel size)
            nn.Conv2d(3,16,5),
            
            # Non-linearity function
            nn.ReLU(inplace=True),
            
            # Pooling Layer
            # MaxPool2d(kernel size)
            nn.MaxPool2d(2,2),
            
            
            nn.Conv2d(16, 32,5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2,2)
        
        )
        
        
        # Fully-Connected Layers
        self.classifier = nn.Sequential(
            nn.Linear(89888, 120),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(120, 84),
            nn.ReLU(inplace=True),
            nn.Linear(84, 3)        
        )
        
        
    
    def forward(self, x):   
        x = self.features(x)
        x = x.view(-1, 89888)
        x = self.classifier(x)

        return x
    
net = Net()

## Eğitim fonksiyonu

In [ ]:
def train(net, criterion, optimizer, scheduler, epochs):
    
    for epoch in range(epochs):
        print('epoch {}/{}'.format(epoch, epochs - 1))

        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                net.train() 
            else:
                net.eval()

            running_loss = 0.0
            running_corrects = 0
            
            num_of_iteration = len(dataloaders[phase])
            
            for co, (inputs, labels) in enumerate(dataloaders[phase]):
                print('{}/{}'.format(co, num_of_iteration))
                
                # Kümülatif toplanan grad'ları temizle
                optimizer.zero_grad()

                # forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    _, preds = torch.max(outputs, 1)
                    
                    loss = criterion(outputs, labels)

                    # Backward pass
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

    return net

## Loss ve Optimizasyon Fonksiyonları

In [ ]:
import torch.optim as optim
from torch.optim import lr_scheduler

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


# Her 10 epoch'da bir learning rate'i 0.1 oranına çarp
# Doc: https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

## Eğitim 

In [ ]:
net = train(net, criterion, optimizer, scheduler,
                       epochs=15)

## Katman Dondurma

In [ ]:
for param in net.features.parameters():
    param.requires_grad = False

criterion = nn.CrossEntropyLoss()

# Sadece fully-connected katmanlar eğitiliyor.
optimizer = optim.SGD(net.classifier.parameters(), lr=0.001, momentum=0.9)

# Her 10 epoch'da bir learning rate'i 0.1 oranına çarp
scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# Doc: https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate


In [ ]:
net = train(net, criterion, optimizer,
                         scheduler, epochs=15)

## Önceden eğitilmiş model kullanma

In [ ]:
net = models.resnet18(pretrained=True)
num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, 3)

In [ ]:
net = train(net, criterion, optimizer, scheduler,
                       epochs=15)